### Database connection

In [1]:
from langchain_community.utilities import SQLDatabase

In [2]:
db_path = '../data/inventory.db'
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
print("dialect:", db.dialect)
print("tables:", db.get_usable_table_names())

dialect: sqlite
tables: ['Orders', 'Stock']


In [3]:
records = db.run("SELECT * FROM Stock LIMIT 1")
print(records)

[('1009AA', 7210.0, 'Kg', 30, 48, 28.76326)]


In [4]:
info = db.get_table_info(["Stock"])
print(info)


CREATE TABLE "Stock" (
	id TEXT, 
	current_stock_quantity FLOAT, 
	units TEXT, 
	avg_lead_time_days BIGINT, 
	maximum_lead_time_days BIGINT, 
	unit_price FLOAT
)

/*
3 rows from Stock table:
id	current_stock_quantity	units	avg_lead_time_days	maximum_lead_time_days	unit_price
1009AA	7210.0	Kg	30	48	28.76326
1077CA	46516.0	Kg	45	70	22.9777
1083AA	48210.0	Kg	45	68	29.02
*/


In [5]:
records = db.run("SELECT * FROM Orders LIMIT 1")
print(records)

[('2022-11-19 00:00:00.000000', '3084CA', 1.0)]


In [6]:
info = db.get_table_info(["Orders"])
print(info)



CREATE TABLE "Orders" (
	date DATETIME, 
	id TEXT, 
	quantity FLOAT
)

/*
3 rows from Orders table:
date	id	quantity
2022-11-19 00:00:00	3084CA	1.0
2022-06-30 00:00:00	2621AA	20.0
2023-04-19 00:00:00	1193BA	147.0
*/


### LLM

In [8]:
from langchain_ollama import ChatOllama

#https://ollama.com/library/qwen2.5-coder:0.5b

llm = ChatOllama(model="qwen2.5-coder:0.5b")

### Tools

In [14]:
# tools
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
toolkit

SQLDatabaseToolkit(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001A7C24B0980>, llm=ChatOllama(model='qwen2.5-coder:latest'))

### SQL agent

In [20]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db,agent_type="openai-tools",verbose=True)


#### Example 1

In [21]:
#expected_query = "SELECT COUNT(*) FROM Orders"
#expected result = 33919
query = "How many orders are there in the Orders table?"
response = agent_executor.invoke(query)
print("response:", response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Orders, Stock
Invoking: `sql_db_schema` with `{'table_names': 'Orders, Stock'}`



CREATE TABLE "Orders" (
	date DATETIME, 
	id TEXT, 
	quantity FLOAT
)

/*
3 rows from Orders table:
date	id	quantity
2022-11-19 00:00:00	3084CA	1.0
2022-06-30 00:00:00	2621AA	20.0
2023-04-19 00:00:00	1193BA	147.0
*/


CREATE TABLE "Stock" (
	id TEXT, 
	current_stock_quantity FLOAT, 
	units TEXT, 
	avg_lead_time_days BIGINT, 
	maximum_lead_time_days BIGINT, 
	unit_price FLOAT
)

/*
3 rows from Stock table:
id	current_stock_quantity	units	avg_lead_time_days	maximum_lead_time_days	unit_price
1009AA	7210.0	Kg	30	48	28.76326
1077CA	46516.0	Kg	45	70	22.9777
1083AA	48210.0	Kg	45	68	29.02
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT COUNT(*) FROM Orders;'}`


SELECT COUNT(*) FROM Orders;
Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(*) FROM Orders;'}`


[(33919,)]There are 33,919 orders in the Orders 

Resultado: Correcto.

La latencia es de 9 minutos y 57s.

In [25]:
db.run('SELECT COUNT(*) FROM Orders;')

'[(33919,)]'

#### Example 2

In [ ]:
expected_query= "SELECT * FROM Orders WHERE quantity = 475"
# Example 2
query2 = "Select the orders that have a quantity equal 475 in the Orders table."
response2 = agent_executor.invoke(query2)
print("response:", response2)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Orders, Stock
Invoking: `sql_db_schema` with `{'table_names': 'Orders'}`



CREATE TABLE "Orders" (
	date DATETIME, 
	id TEXT, 
	quantity FLOAT
)

/*
3 rows from Orders table:
date	id	quantity
2022-11-19 00:00:00	3084CA	1.0
2022-06-30 00:00:00	2621AA	20.0
2023-04-19 00:00:00	1193BA	147.0
*/Now I can see the schema of the "Orders" table and understand that it contains columns for "date", "id", and "quantity". 

To find orders with a quantity equal to 475, I will construct an SQL query to select these rows from the "Orders" table.

```sql
SELECT * FROM Orders WHERE quantity = 475;
```

I should check if this query is correct before executing it.



> Finished chain.
response: {'input': 'Select the orders that have a quantity equal 475 in the Orders table.', 'output': 'Now I can see the schema of the "Orders" table and understand that it contains columns for "date", "id", and "quantity". \n\nTo find o

Resultado:  Ha devuelto la query correcta. 
SELECT * FROM Orders WHERE quantity = 475

La latencia es 1min 57s.

In [24]:
db.run("SELECT * FROM Orders WHERE quantity = 475")

"[('2023-04-16 00:00:00.000000', '2785BA', 475.0), ('2022-05-23 00:00:00.000000', '1993AA', 475.0), ('2022-05-23 00:00:00.000000', '2727CA', 475.0), ('2022-04-03 00:00:00.000000', '2814BA', 475.0), ('2022-09-23 00:00:00.000000', '2847BA', 475.0), ('2022-08-20 00:00:00.000000', '2291AA', 475.0), ('2022-03-28 00:00:00.000000', '2803BA', 475.0)]"

#### Example 3

In [26]:
expected_query= "SELECT id, date, quantity FROM Orders WHERE quantity = 475 LIMIT 10;"
expected_result=[('2785BA', '2023-04-16 00:00:00.000000', 475.0), 
                 ('1993AA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2727CA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2814BA', '2022-04-03 00:00:00.000000', 475.0), 
                 ('2847BA', '2022-09-23 00:00:00.000000', 475.0), 
                 ('2291AA', '2022-08-20 00:00:00.000000', 475.0), 
                 ('2803BA', '2022-03-28 00:00:00.000000', 475.0)]

# Example 2
query3 = "What are the first 10 orders that have a quantity equal 475 in the Orders table."
response3 = agent_executor.invoke(query3)
print("response:", response3)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Orders, Stock
Invoking: `sql_db_query_checker` with `{'query': 'SELECT order_id, quantity, order_date FROM Orders WHERE quantity = 475 LIMIT 10'}`


```sql
SELECT order_id, quantity, order_date FROM Orders WHERE quantity = 475 LIMIT 10;
```
Invoking: `sql_db_query` with `{'query': 'SELECT order_id, quantity, order_date FROM Orders WHERE quantity = 475 LIMIT 10;'}`


Error: (sqlite3.OperationalError) no such column: order_id
[SQL: SELECT order_id, quantity, order_date FROM Orders WHERE quantity = 475 LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)The error indicates that the `order_id` column does not exist in the `Orders` table. I should double-check the schema of the `Orders` table to find out what columns are available.

Let's query the schema of the `Orders` table first.

```sql
SELECT * FROM sql_db_schema('Orders');
```

This will provide me with the correct column names

Resultado: 
Ha fallado porque ha utilizado un campo (order_id) que no existe.

### Example 4 (CoT + schema)

In [27]:
expected_query= "SELECT id, date, quantity FROM Orders WHERE quantity = 475 LIMIT 10;"
expected_result=[('2785BA', '2023-04-16 00:00:00.000000', 475.0), 
                 ('1993AA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2727CA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2814BA', '2022-04-03 00:00:00.000000', 475.0), 
                 ('2847BA', '2022-09-23 00:00:00.000000', 475.0), 
                 ('2291AA', '2022-08-20 00:00:00.000000', 475.0), 
                 ('2803BA', '2022-03-28 00:00:00.000000', 475.0)]

# Example 2
query4 = "What are records that have a quantity equal 475 in the Orders table. The table Orders has the columns: id, date and quantity. Let's think step by step."
response4 = agent_executor.invoke(query4)
print("response:", response4)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Orders, Stock
Invoking: `sql_db_schema` with `{'table_names': 'Orders'}`



CREATE TABLE "Orders" (
	date DATETIME, 
	id TEXT, 
	quantity FLOAT
)

/*
3 rows from Orders table:
date	id	quantity
2022-11-19 00:00:00	3084CA	1.0
2022-06-30 00:00:00	2621AA	20.0
2023-04-19 00:00:00	1193BA	147.0
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT * FROM Orders WHERE quantity = 475'}`


```sql
SELECT * FROM Orders WHERE quantity = 475;
```I should execute the query to get the records.



> Finished chain.
response: {'input': "What are records that have a quantity equal 475 in the Orders table. The table Orders has the columns: id, date and quantity. Let's think step by step.", 'output': 'I should execute the query to get the records.\n\n'}


Resultado: Correcto. Devuelve la query aunque no la ejecuta. SELECT * FROM Orders WHERE quantity = 475

Parece que ignora el resto de los comentarios sobre el schema.

Latencia 7min y 21s.



### Example 5 (Schema)

In [28]:
expected_query= "SELECT id, date, quantity FROM Orders WHERE quantity = 475 LIMIT 10;"
expected_result=[('2785BA', '2023-04-16 00:00:00.000000', 475.0), 
                 ('1993AA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2727CA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2814BA', '2022-04-03 00:00:00.000000', 475.0), 
                 ('2847BA', '2022-09-23 00:00:00.000000', 475.0), 
                 ('2291AA', '2022-08-20 00:00:00.000000', 475.0), 
                 ('2803BA', '2022-03-28 00:00:00.000000', 475.0)]


query5 = "What are records that have a quantity equal 475 in the Orders table. The table Orders has the columns: id, date and quantity."
response5 = agent_executor.invoke(query5)
print("response:", response5)



> Entering new SQL Agent Executor chain...


> Finished chain.
response: {'input': 'What are records that have a quantity equal 475 in the Orders table. The table Orders has the columns: id, date and quantity.', 'output': ''}


Resultado: Fallo. No hace nada.

### Example 6 (CoT)

In [29]:
expected_query= "SELECT id, date, quantity FROM Orders WHERE quantity = 475 LIMIT 10;"
expected_result=[('2785BA', '2023-04-16 00:00:00.000000', 475.0), 
                 ('1993AA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2727CA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2814BA', '2022-04-03 00:00:00.000000', 475.0), 
                 ('2847BA', '2022-09-23 00:00:00.000000', 475.0), 
                 ('2291AA', '2022-08-20 00:00:00.000000', 475.0), 
                 ('2803BA', '2022-03-28 00:00:00.000000', 475.0)]


query6 = "What are records that have a quantity equal 475 in the Orders table. Let's think step by step."
response6 = agent_executor.invoke(query6)
print("response:", response6)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Orders, Stock
Invoking: `sql_db_schema` with `{'table_names': 'Orders'}`



CREATE TABLE "Orders" (
	date DATETIME, 
	id TEXT, 
	quantity FLOAT
)

/*
3 rows from Orders table:
date	id	quantity
2022-11-19 00:00:00	3084CA	1.0
2022-06-30 00:00:00	2621AA	20.0
2023-04-19 00:00:00	1193BA	147.0
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT * FROM Orders WHERE quantity = 475 LIMIT 10'}`


```sql
SELECT * FROM Orders WHERE quantity = 475 LIMIT 10
```I don't know

> Finished chain.
response: {'input': "What are records that have a quantity equal 475 in the Orders table. Let's think step by step.", 'output': "I don't know"}


Resultado: Devuelve la query correcta pero no es capaz de ejecutarlo.

Latencia 4minutos y 50s